In [2]:
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import service_pb2_grpc

stub = service_pb2_grpc.V2Stub(ClarifaiChannel.get_grpc_channel())

import pandas as pd
import numpy as np
import time

In [7]:
import zipfile

with zipfile.ZipFile("food.zip","r") as zip_ref:
    zip_ref.extractall("food_porn")



In [8]:
dir = 'food_porn/food/'

In [17]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [18]:
from clarifai_grpc.grpc.api import service_pb2, resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2

metadata = (('authorization', 'Key 0bec95f3015f4a26b5edd2cb62a4fdc6'),)

def define_food (number, dir):
    with open(dir + number + '.jpg', "rb") as f:
        file_bytes = f.read()

    request = service_pb2.PostModelOutputsRequest(
        model_id='bd367be194cf45149e75f01d59f77ba7',
        inputs=[
        resources_pb2.Input(data=resources_pb2.Data(image=resources_pb2.Image(base64=file_bytes)))
        ])
    response = stub.PostModelOutputs(request, metadata=metadata)

    if response.status.code != status_code_pb2.SUCCESS:
        raise Exception("Request failed, status code: " + str(response.status.code))

    temp = [[concept.name, concept.value] for concept in response.outputs[0].data.concepts]
    return flatten_list(temp)

In [19]:
print(define_food('00005', dir))

['pie', 0.9858543276786804, 'pastry', 0.9853608012199402, 'cake', 0.9836643934249878, 'sweet', 0.9713603854179382, 'bread', 0.9693325161933899, 'cheese', 0.782551646232605, 'dairy product', 0.7273536920547485, 'meat', 0.682706356048584, 'cookie', 0.5992613434791565, 'crust', 0.5958021283149719, 'vegetable', 0.5746768712997437, 'dough', 0.5121260285377502, 'chocolate', 0.45739659667015076, 'pudding', 0.3970496952533722, 'cream', 0.36376792192459106, 'bread pudding', 0.3283917307853699, 'pork pie', 0.31448814272880554, 'tomato', 0.3140222430229187, 'meat pie', 0.29785072803497314, 'sauce', 0.28030264377593994]


In [23]:
result = pd.DataFrame(np.chararray((0,40)))

for i in range(0, 10000):
    time.sleep(0.102)
    number = '{0:05}'.format(i)
    print('Currently at {}'.format(i))
    try:
        food = define_food(number, dir)
    except:
        food = [0 for x in range(0,40)]

    temp = pd.DataFrame(define_food(number, dir))
    
    result = pd.concat((result, temp.transpose()))

result.to_csv('classification2.csv', header = False, index = False)

Currently at 0


Exception: Request failed, status code: 10020

In [22]:
result.to_csv('classification.csv', header = False, index = False)

In [122]:
print(result)

           0          1          2        3          4       5          6   \
0        fish    seafood  vegetable    water       herb   grass   samphire   
0        cake      sweet     pastry      pie      bread  cookie      candy   
0        cake     cookie  chocolate   pastry      cream   sweet      candy   
0       sweet      bread      wheat     cake     pastry  cereal  chocolate   
0       sauce       meat      sweet  chicken  vegetable  tomato       pork   
..        ...        ...        ...      ...        ...     ...        ...   
0   cranberry  blueberry  raspberry    lemon  chocolate   sweet     almond   
0   chocolate     cookie     muffin    sweet     pastry    cake  blueberry   
0       sweet    sausage      bread     meat     pastry  cookie       cake   
0       pizza      salad     cheese  avocado   zucchini   onion      bacon   
0       water        tea      candy      ice      sweet    cake     tongue   

            7           8          9              10           

In [ ]:
import csv
from google.protobuf.struct_pb2 import Struct
import time

from clarifai_grpc.grpc.api import service_pb2_grpc, service_pb2, resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel

channel = ClarifaiChannel.get_insecure_grpc_channel()

stub = service_pb2_grpc.V2Stub(channel)

tic = time.time()

file_address = 'PATH_TO_CSV_FILE'

metadata = (('authorization', 'Key 0bec95f3015f4a26b5edd2cb62a4fdc6'),)


with open(file_address, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    inputs = []
    count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        input_metadata = Struct()



        ## You can update as many metadatafields as you and name the work {{metadata_fieldx}} to whatever you need

        input_metadata.update(
            {"metadata_field1": row["field1"], "metadata_field2": row["field2"], "metadata_field3": row["field3"],
             })

        inputs.append(
            resources_pb2.Input(
                ### You can replace id below with one of the values from your csv file or you can delete this
                ###option
                model_id='bd367be194cf45149e75f01d59f77ba7',
                data=resources_pb2.Data(
                    image=resources_pb2.Image(base64=file_bytes),
                    metadata=input_metadata
                )
            )
        )
        if len(inputs) == 32:
            post_input_results_response = stub.PostInputs(
                service_pb2.PostInputsRequest(inputs=inputs),
                metadata=metadata
            )
            inputs = []
        count += 1
        if count % 10000 == 0:
            time.sleep(1)
        print(count, ' batches of 32 processed')
        # print(32 * count, 'images processed\n\n')

toc = time.time()

hours, rem = divmod(toc - tic, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))